In [41]:
from random import random, choice, randint
from functools import reduce
from collections import namedtuple
from copy import copy

import numpy as np

def goal_check(state, sets):
    return np.all(reduce(
        np.logical_or,
        [sets[i] for i, t in enumerate(state) if t],
        np.array([False for _ in range(len(sets[0]))]),
    ))

def fitness(state, sets):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [sets[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(len(sets[0]))]),
        )
    )
    return valid, -cost

def tweak(state, sets, lambda_value):
    best_state = state
    
    for _ in range(lambda_value):
        new_state = copy(state)
        index = randint(0, len(sets[0]) - 1)
        new_state[index] = not new_state[index]
        
        if fitness(new_state, sets) >= fitness(state, sets):
            best_state = new_state

    return best_state

In [42]:
num_points = [100, 1_000, 5_000]
density = [.3, .7]

for npoints in num_points:
    for d in density:
        NUM_SETS = npoints
        SETS = tuple(np.array([random() < d for _ in range(npoints)]) for _ in range(NUM_SETS))
        State = namedtuple('State', ['taken', 'not_taken'])
                
        current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
        
        lambda_value = 100
        
        for step in range(100_000):
            new_state = tweak(current_state, SETS, lambda_value)
            if fitness(new_state, SETS) >= fitness(current_state, SETS):
                current_state = new_state
            if goal_check(current_state, SETS):
                break
        
        num_set = sum(1 for value in current_state if value)
        print(f"num_points: {npoints}, density: {d}, taken sets: {num_set}")

        # Ho deciso di mettere lamda abbastanza alto cosi da cercare il lae soluzioni 
        # migliori con il tweak e poi stoppare appena trovo la prima soluzione ammissibile
        # in questo modo il programma dovrebbe dare un risultato buono in poco tempo

num_points: 100, density: 0.3, taken sets: 12
num_points: 100, density: 0.7, taken sets: 5
num_points: 1000, density: 0.3, taken sets: 19
num_points: 1000, density: 0.7, taken sets: 6
num_points: 5000, density: 0.3, taken sets: 21
num_points: 5000, density: 0.7, taken sets: 7
